


Cross Validation & Naive Bayes Lab - SMS Spam Classification
===============
* orignally developed by Ankit Jain
* modified by Justin Breucop
* modified by Dylan Hercher

Data source: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

# Section 1)

## Cross Validation from Scratch

Let's build it the function together! The steps to cross validation are:
1. Randomly separate your training set into _k_ groups
2. For each group _k_:
>1. Train your model on the other groups
>2. Score your model using group _k_ as validation
>3. Save your score and move to your next group

3. Add your _k_ scores and divide by _k_ to get your average score

In [1]:
# Importing Packages 
import numpy as np
import pandas as pd

from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split

In [195]:
model = KNeighborsClassifier(10)

df = pd.DataFrame.from_csv('../data/titanic-train.csv',index_col=None)[['Age','Pclass','SibSp','Survived']].dropna()
to_predict = "Survived"
features=['Age','Pclass','SibSp']
data = df[features]
label = df[to_predict]
folds=5

In [199]:
label

0      0
1      1
2      1
3      1
4      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     0
15     1
16     0
18     0
20     0
21     1
22     1
23     1
24     0
25     1
27     0
30     0
33     0
34     0
35     0
37     0
38     0
      ..
856    1
857    1
858    1
860    0
861    0
862    1
864    0
865    1
866    1
867    0
869    1
870    0
871    1
872    0
873    0
874    1
875    1
876    0
877    0
879    1
880    1
881    0
882    0
883    0
884    0
885    0
886    0
887    1
889    1
890    0
Name: Survived, dtype: int64

Let's build in class!

[0, 1, 2, 3, 4]

In [196]:
#Function here:


def cross_validate_df(data,label,model,k):
    scores = []
    """Return Average score across k iterations
    Parameters
    -----------
    data : DataFrame with features to use in X
    label : Series with target y
    model : ML Model to use
    k : int number of iterations
    """
    # TODO: Design the cross validation function
    # it may be helpful to try writing the function with k=1 first
    
    shuffle = data.reindex(np.random.permutation(data.index))
    shuffle.reset_index(inplace=True,drop=True)
    shuffle.head()
    
    print shuffle.head()
    
    fold_list = []
    fold_labels_list = []
    scores = []

    for i in range(k):
        fold_list.append(shuffle.values[i*((len(df)/folds)):(len(df)/folds)*(i+1),1])
        fold_labels_list.append(label[i*((len(df)/folds)):(len(df)/folds)*(i+1),0])
    
    for i in range(k):    
        train_slice = []
        train_values_slice = []
        fold_slice = fold_list[i]
        fold_labels_slice = fold_labels_list[i]
        remaining = range(5)
        remaining.remove(int(i))
        for j in remaining:
            train_slice = np.concatenate((train_slice,fold_list[j]))
            train_values_slice = np.concatenate((train_values_slice,fold_labels_list[j]))
        print "*******train********"
        print train_slice
        print "********values******"
        print train_values_slice
        
        model.fit(train_slice,train_values_slice)    
        sc = model.score(fold_slice, fold_labels_slice)
        scores.append(sc)
        

    return scores

In [197]:
print cross_validate_df(data,label,model,folds)

   Age  Pclass  SibSp
0    4       3      0
1   33       1      0
2   33       1      1
3   33       2      0
4   32       2      1
*******train********
[ 3.  3.  3.  2.  1.  3.  3.  3.  3.  2.  3.  3.  2.  3.  1.  2.  3.  2.
  3.  2.  3.  3.  2.  3.  3.  1.  3.  3.  3.  3.  1.  1.  1.  3.  2.  3.
  3.  2.  1.  1.  3.  1.  3.  3.  3.  2.  1.  1.  2.  1.  2.  3.  2.  1.
  3.  1.  2.  2.  2.  2.  2.  1.  1.  3.  1.  3.  2.  1.  1.  2.  3.  1.
  3.  3.  2.  3.  1.  3.  2.  1.  3.  3.  1.  3.  3.  3.  3.  2.  3.  3.
  3.  2.  1.  3.  2.  2.  1.  3.  3.  3.  3.  3.  1.  2.  1.  1.  3.  1.
  3.  3.  1.  1.  3.  3.  3.  1.  1.  3.  2.  3.  3.  3.  3.  3.  1.  3.
  3.  2.  3.  3.  1.  2.  2.  1.  3.  3.  3.  1.  3.  2.  1.  2.  2.  3.
  3.  3.  3.  3.  1.  3.  3.  1.  2.  1.  1.  3.  2.  2.  1.  1.  3.  1.
  3.  3.  2.  2.  2.  2.  3.  3.  2.  3.  1.  2.  1.  2.  3.  3.  3.  3.
  2.  3.  3.  2.  2.  3.  3.  2.  3.  1.  1.  3.  3.  1.  3.  3.  3.  3.
  2.  3.  3.  2.  3.  2.  3.  3.  2.  1.  3.

ValueError: Found arrays with inconsistent numbers of samples: [  1 568]

In [188]:

fold_list = []
fold_labels_list = []

for i in range(folds):
    fold_list.append(shuffle.values[i*((len(df)/folds)):(len(df)/folds)*(i+1),1])
    fold_labels_list.append(df[to_predict].ix[i*((len(df)/folds)):(len(df)/folds)*(i+1),0])

for i in range(folds):    
    train_slice = []
    train_values_slice = []
    fold_slice = fold_list[i]
    fold_labels_slice = fold_labels_list[i]
    remaining = range(5)
    remaining.remove(int(i))
    for j in remaining:
        train_slice = np.concatenate((train_slice,fold_list[j]))
        train_values_slice = np.concatenate((train_values_slice,fold_labels_list[j]))
        
        
print len(fold_labels_list[3])
    
    
    


1114


In [119]:
fold_list = []

for i in range(folds):
    fold_list.append(shuffle.values[i*((len(df)/folds)):(len(df)/folds)*(i+1),1])

fold_list[2]
    


array(['o turns out i had stereo love on mi phone under the unknown album.',
       'Not for possession, especially not first offense',
       "You should know now. So how's anthony. Are you bringing money. I've school fees to pay and rent and stuff like that. Thats why i need your help. A friend in need....|",
       ...,
       'V-aluable. A-ffectionate. L-oveable. E-ternal. N-oble. T-ruthful. I-ntimate. N-atural. E-namous. Happy "VALENTINES DAY" in advance',
       'Beautiful Truth against Gravity.. Read carefully: "Our heart feels light when someone is in it.. But it feels very heavy when someone leaves it.." GOOD NIGHT',
       'Ok i go change also...'], dtype=object)

In [135]:
x = range(5)
x.remove(1)
print x

[0, 2, 3, 4]


In [105]:
len(shuffle.values[2*((len(df)/folds)):(len(df)/folds)*(2+1)])

1114

5570

In [107]:
(len(df)/folds)*(4+1)

5570

In [ ]:
X = data.values
        y = label.values
        
        X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=(1.0/folds))
        knn = model.fit(X_train,y_train)
        score = knn.score(X_test, y_test)
        scores.append(score)


In [20]:
# Once Complete test your function here
cross_validate_df(data,label,model,5)

ValueError: Found array with 0 sample(s) (shape=(0, 3)) while a minimum of 1 is required.

### Built-in Cross Validation
There is also a very simple cross validation function provided by sklearn

In [200]:
from sklearn.cross_validation import cross_val_score

# Results of using the built-in cross validation
# Note: Default is Kfold, but within sklearn.cross_validation
# there are many types of validation that can be used
cross_val_score(model, data, label, cv=5)

array([ 0.64335664,  0.69230769,  0.6993007 ,  0.74125874,  0.67605634])

In [201]:
cross_val_score(model, data, label, cv=5)

array([ 0.64335664,  0.69230769,  0.6993007 ,  0.74125874,  0.67605634])

In [330]:
# Applying 5 fold cross-validation to our model

np.mean(cross_val_score(model, data, label, cv=5))

0.6904560228503891

# Section 2)
## Naive Bayes and SMS Spam Classification

In [209]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics


In [210]:
## READING IN THE DATA
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

In [211]:
# examine the data
df.head()

,label,msg
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [212]:
df[df.label=='spam'].head()

,label,msg
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [213]:
df.label.value_counts()

ham     4825
spam     747
dtype: int64

In [214]:
df.msg.describe()

count                       5572
unique                      5169
top       Sorry, I'll call later
freq                          30
Name: msg, dtype: object

In [215]:
# Convert the label into a binary variable
# Remember the map function we learned before?
df['label'] = df.label.map({'ham': 0 , 'spam':1})

In [216]:
df.head()

,label,msg
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [217]:
# split into training and testing sets by calling sklearn lib
# by default, the data set is split into 0.75 (training) and 0.25 (testing)
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

In [12]:
print X_train.shape
print X_train

(4179,)
710     4mths half price Orange line rental & latest c...
3740                           Did you stitch his trouser
2711    Hope you enjoyed your new content. text stop t...
3155    Not heard from U4 a while. Call 4 rude chat pr...
3748    Ü neva tell me how i noe... I'm not at home in...
2389    wiskey Brandy Rum Gin Beer Vodka Scotch Shampa...
3464    i am seeking a lady in the street and a freak ...
772     Lol! U drunkard! Just doing my hair at d momen...
3667    I'm turning off my phone. My moms telling ever...
4955    U coming back 4 dinner rite? Dad ask me so i r...
854     AH POOR BABY!HOPE URFEELING BETTERSN LUV! PROB...
4079                  Gam gone after outstanding innings.
2837                         Nice.nice.how is it working?
1392                  Haha just kidding, papa needs drugs
5533    Hey chief, can you give me a bell when you get...
874     Ugh its been a long day. I'm exhausted. Just w...
4408    Awesome, plan to get here any time after like ...
3990  

In [13]:
X_test.shape

(1393,)

Now we need to convert the text into feature vectors which can be used for machine learning purposes.
We will use the scikit function of CountVectorizer to 'convert text into a matrix of token counts'

 http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#### Lets try a simple example

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
# start with a simple example
train_simple = ['call you tonight',
                'Call me a cab',
                'please call me... PLEASE!']

In [16]:
# learn the 'vocabulary' of the training data
#remove meaningless words like 'a', 'the', 'at', etc

vect = CountVectorizer(decode_error='ignore')
vect.fit(train_simple)
vect.get_feature_names()

[u'cab', u'call', u'me', u'please', u'tonight', u'you']

In [17]:
# transform training data into a 'document-term matrix'
train_simple_dtm = vect.transform(train_simple)
train_simple_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]])

In [ ]:
# We can see how we've adjusted our data easily!
# examine the vocabulary and document-term matrix together
print train_simple
    
pd.DataFrame(train_simple_dtm.toarray(), columns=vect.get_feature_names())

In [18]:
# transform testing data into a document-term matrix (using existing vocabulary)
test_simple = ["please don't call me"]
test_simple_dtm = vect.transform(test_simple)
test_simple_dtm.toarray()

pd.DataFrame(test_simple_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


In [33]:
print test_simple_dtm

  (0, 1)	1
  (0, 2)	1
  (0, 3)	1


#### Question:  How does the above test_simple show how things can go wrong?

#### Exercise: Using the dataset below
   * Vectorize the text
   * Store the results in a DataFrame
   * Show word counts (hint: one dataframe describer can do this)
   * Transform the test text

In [20]:
train_exp = ['where is my taco?',
                'did I eat the taco',
                'I can easily eat my way through that whole box of tacos!',
                'I think way too much about tacos, huh',
                'taco, taco, taco!!!'                
               ]
test_exp = [
    'where did he go?', 'how long did the whole thing last', 'lets go eat one taco or multiple tacos'
]

In [21]:
vect = CountVectorizer(decode_error='ignore')
vect.fit(train_exp)
vect.get_feature_names()

[u'about',
 u'box',
 u'can',
 u'did',
 u'easily',
 u'eat',
 u'huh',
 u'is',
 u'much',
 u'my',
 u'of',
 u'taco',
 u'tacos',
 u'that',
 u'the',
 u'think',
 u'through',
 u'too',
 u'way',
 u'where',
 u'whole']

In [206]:
train_exp_dtm = vect.transform(train_exp)


pd.DataFrame(train_exp_dtm.toarray(), columns=vect.get_feature_names())

train_exp_dtm.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [29]:
test_exp_dtm = vect.transform(test_exp)
test_exp_dtm.toarray()

train_exp_df = pd.DataFrame(test_exp_dtm.toarray(), columns=vect.get_feature_names())
train_exp_df

,about,box,can,did,easily,eat,huh,is,much,my,...,taco,tacos,that,the,think,through,too,way,where,whole
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0


## Vectorizing our SMS Dataset

In [218]:
# instantiate the vectorizer ( use variable name as vect)
vect = CountVectorizer(decode_error = 'ignore')
vect.fit(X_train)
vect.get_feature_names()

[u'00',
 u'000',
 u'008704050406',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07090201529',
 u'07090298926',
 u'07123456789',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705',
 u'08000938767',
 u'08001950382',
 u'08002888812',
 u'08002986030',
 u'08002986906',
 u'08002988890',
 u'08006344447',
 u'0808',
 u'08081263000',
 u'08081560665',
 u'0825',
 u'083',
 u'0844',
 u'08448714184',
 u'0845',
 u'08450542832',
 u'08452810071',
 u'08452810073',
 u'08452810075over18',


In [207]:
# transform testing data into a document-term matrix: Use Variable name as test_dtm
train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)
print train_dtm

  (0, 58)	1
  (0, 321)	1
  (0, 601)	1
  (0, 1828)	1
  (0, 1849)	1
  (0, 2371)	1
  (0, 3373)	2
  (0, 3721)	1
  (0, 3733)	1
  (0, 4551)	1
  (0, 4657)	1
  (0, 5121)	1
  (0, 5453)	1
  (0, 5559)	1
  (0, 5606)	1
  (0, 5608)	1
  (0, 5849)	1
  (0, 5853)	1
  (0, 6094)	1
  (0, 6428)	1
  (0, 7806)	1
  (0, 8091)	1
  (0, 8674)	1
  (1, 2606)	1
  (1, 3888)	1
  :	:
  (4177, 3226)	1
  (4177, 3276)	1
  (4177, 4253)	1
  (4177, 4335)	1
  (4177, 4376)	1
  (4177, 4987)	1
  (4177, 5199)	1
  (4177, 5276)	1
  (4177, 5604)	1
  (4177, 5794)	1
  (4177, 6337)	1
  (4177, 6442)	1
  (4177, 6638)	1
  (4177, 6811)	1
  (4177, 7093)	1
  (4177, 7644)	1
  (4177, 7716)	1
  (4177, 7806)	1
  (4177, 7813)	1
  (4177, 8086)	1
  (4177, 8492)	1
  (4178, 1979)	1
  (4178, 4968)	1
  (4178, 7052)	1
  (4178, 8492)	1


In [219]:
# Get the length and names of the feature names
train_features = vect.get_feature_names()
len(train_features)

7456

In [34]:
train_features[:50]

[u'00',
 u'000',
 u'008704050406',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07090201529',
 u'07090298926',
 u'07123456789',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705']

In [35]:
train_features[-50:]

[u'yer',
 u'yes',
 u'yest',
 u'yesterday',
 u'yet',
 u'yetunde',
 u'yijue',
 u'ym',
 u'ymca',
 u'yo',
 u'yoga',
 u'yogasana',
 u'yor',
 u'yorge',
 u'you',
 u'youdoing',
 u'youi',
 u'youphone',
 u'your',
 u'youre',
 u'yourjob',
 u'yours',
 u'yourself',
 u'youwanna',
 u'yowifes',
 u'yoyyooo',
 u'yr',
 u'yrs',
 u'ything',
 u'yummmm',
 u'yummy',
 u'yun',
 u'yunny',
 u'yuo',
 u'yuou',
 u'yup',
 u'zac',
 u'zaher',
 u'zealand',
 u'zebra',
 u'zed',
 u'zeros',
 u'zhong',
 u'zindgi',
 u'zoe',
 u'zoom',
 u'zouk',
 u'zyada',
 u'\xe8n',
 u'\u3028ud']

In [36]:
# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:

# Revisit Numpy
arr = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
print arr[0, 0]
print arr[1, 3]
print arr[0, :]
print arr[:, 0]
print np.sum(arr)
print np.sum(arr,axis = 0)
print np.sum(arr,axis = 1)




1
8
[1 2 3 4]
[1 5]
36
[ 6  8 10 12]
[10 26]


In [38]:
# exercise: calculate the number of tokens in the 0th message in train_arr
print np.sum(train_arr[0,:])

24


In [39]:

# exercise: count how many times the 0th token appears across ALL messages in train_arr
print np.sum(train_arr[:,0])

5


In [224]:
# exercise: count how many times EACH token appears across ALL messages in train_arr
print np.sum(train_arr, axis=0)

[ 5 23  2 ...,  1  1  1]


In [254]:
counts = pd.DataFrame(np.sum(train_arr, axis=0),columns = ['counts'])
words.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9, 
            ...
            7446, 7447, 7448, 7449, 7450, 7451, 7452, 7453, 7454, 7455],
           dtype='int64', length=7456)

In [257]:
# exercise: create a DataFrame of tokens with their counts.


words = pd.DataFrame(train_features,columns = ['word'])
    
count_df = pd.merge(words,counts,left_index=True,right_index=True)

count_df.head()


,word,counts
0,00,5
1,000,23
2,008704050406,2
3,0121,1
4,01223585236,1


In [46]:
train_arr[:]

array([[0, 0, 0, ..., 0, 0, 0]])

### Let's build the model with Naive Bayes Now

http://scikit-learn.org/stable/modules/naive_bayes.html

In [48]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [49]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm)
preds

array([0, 0, 0, ..., 0, 1, 0])

In [262]:
np.sum(preds)

179

In [51]:
# compare predictions to true labels
from sklearn import metrics
print metrics.accuracy_score(y_test, preds)
print metrics.confusion_matrix(y_test, preds)

# confusion matrix: http://en.wikipedia.org/wiki/Confusion_matrix

0.988513998564
[[1203    5]
 [  11  174]]


In [52]:
# exercise: show the message text for the false positives 
X_test[(y_test == 0) & (preds == 1)]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: msg, dtype: object

In [53]:
# exercise: show the message text for the false negatives
X_test[y_test > preds]
# or
X_test[(y_test == 1) & (preds == 0)]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: msg, dtype: object

In [ ]:
## USING ALL DATA AND CROSS-VALIDATION, run NB again


In [266]:
# Here is the condensed code needed for reference
df = pd.DataFrame.from_csv("../data/SMSSpamCollection.tsv",sep='\t',header=0,index_col=None)

df['label'] = df.label.map({'ham': 0 , 'spam':1})

df.msg.values

len(df)


5572

In [56]:
X_train, X_test, y_train, y_test = train_test_split(df.msg, df.label, random_state=1)

vect = CountVectorizer(decode_error = 'ignore')

vect.fit(X_train)
vect.get_feature_names()

train_dtm = vect.transform(X_train)
test_dtm = vect.transform(X_test)

In [308]:
## EXERCISE: CALCULATE THE 'SPAMMINESS' OF EACH TOKEN

# create separate DataFrames for ham and spam ( df_ham and df_spam)

df_ham = df[df.label==0]
df_spam = df[df.label==1]


In [309]:
# learn the vocabulary of ALL messages and save it
vocab_all = vect.fit(df.msg.values)
len(vocab_all.get_feature_names())
len(df.msg.values)

5572

In [310]:
# create document-term matrix of ham, then convert to a regular array

ham_dtm = vect.transform(df_ham.msg.values)
ham_dtm.toarray()

ham_df = pd.DataFrame(ham_dtm.toarray(), columns=vect.get_feature_names())

In [311]:
# create document-term matrix of spam, then convert to a regular array

spam_dtm = vect.transform(df_spam.msg.values)
spam_dtm.toarray()

spam_df = pd.DataFrame(spam_dtm.toarray(), columns=vect.get_feature_names())
spam_df.sum(axis=0)




00              10
000             29
000pes           0
008704050406     2
0089             1
0121             1
01223585236      1
01223585334      2
0125698789       0
02               8
0207             3
02072069400      1
02073162414      2
02085076972      1
021              2
03              13
04              12
0430             1
05               5
050703           2
0578             2
06               8
07               2
07008009200      1
07046744435      1
07090201529      1
07090298926      1
07099833605      1
07123456789      2
0721072          1
                ..
yowifes          0
yoyyooo          0
yr              11
yrs              3
ystrday          0
ything           0
yummmm           0
yummy            0
yun              0
yunny            0
yuo              0
yuou             0
yup              0
yupz             0
zac              0
zaher            0
zealand          0
zebra            1
zed              6
zeros            0
zhong            0
zindgi      

In [312]:
# count how many times EACH token appears across ALL messages in ham_arr
spam_df.sum(axis=0)

00              10
000             29
000pes           0
008704050406     2
0089             1
0121             1
01223585236      1
01223585334      2
0125698789       0
02               8
0207             3
02072069400      1
02073162414      2
02085076972      1
021              2
03              13
04              12
0430             1
05               5
050703           2
0578             2
06               8
07               2
07008009200      1
07046744435      1
07090201529      1
07090298926      1
07099833605      1
07123456789      2
0721072          1
                ..
yowifes          0
yoyyooo          0
yr              11
yrs              3
ystrday          0
ything           0
yummmm           0
yummy            0
yun              0
yunny            0
yuo              0
yuou             0
yup              0
yupz             0
zac              0
zaher            0
zealand          0
zebra            1
zed              6
zeros            0
zhong            0
zindgi      

In [313]:
# count how many times EACH token appears across ALL messages in spam_arr
ham_df.sum(axis=0)
vect.get_feature_names()

[u'00',
 u'000',
 u'000pes',
 u'008704050406',
 u'0089',
 u'0121',
 u'01223585236',
 u'01223585334',
 u'0125698789',
 u'02',
 u'0207',
 u'02072069400',
 u'02073162414',
 u'02085076972',
 u'021',
 u'03',
 u'04',
 u'0430',
 u'05',
 u'050703',
 u'0578',
 u'06',
 u'07',
 u'07008009200',
 u'07046744435',
 u'07090201529',
 u'07090298926',
 u'07099833605',
 u'07123456789',
 u'0721072',
 u'07732584351',
 u'07734396839',
 u'07742676969',
 u'07753741225',
 u'0776xxxxxxx',
 u'07781482378',
 u'07786200117',
 u'077xxx',
 u'078',
 u'07801543489',
 u'07808',
 u'07808247860',
 u'07808726822',
 u'07815296484',
 u'07821230901',
 u'078498',
 u'07880867867',
 u'0789xxxxxxx',
 u'07946746291',
 u'0796xxxxxx',
 u'07973788240',
 u'07xxxxxxxxx',
 u'08',
 u'0800',
 u'08000407165',
 u'08000776320',
 u'08000839402',
 u'08000930705',
 u'08000938767',
 u'08001950382',
 u'08002888812',
 u'08002986030',
 u'08002986906',
 u'08002988890',
 u'08006344447',
 u'0808',
 u'08081263000',
 u'08081560665',
 u'0825',
 u'083',
 

In [314]:
# create a DataFrame of tokens with their separate ham and spam counts
data_ = {'ham_count':ham_df.sum(axis=0),'spam_count':spam_df.sum(axis=0)}


counts_df = pd.DataFrame(data_)
counts_df.head()




,ham_count,spam_count
00,0,10
000,0,29
000pes,1,0
008704050406,0,2
0089,0,1


In [315]:
# add one to ham counts and spam counts so that ratio calculations (below) make more sensse
counts_df['ham_count'] = counts_df['ham_count'] + 1
counts_df['spam_count'] = counts_df['spam_count'] + 1



In [316]:
counts_df.head()

,ham_count,spam_count
00,1,11
000,1,30
000pes,2,1
008704050406,1,3
0089,1,2


In [329]:
# calculate ratio of spam-to-ham for each token

counts_df['spam-to-ham'] = 1.0 * counts_df.spam_count/counts_df.ham_count

counts_df.sort('spam-to-ham',ascending=False)

,ham_count,spam_count,spam-to-ham
claim,1,114,114.000000
prize,1,94,94.000000
150p,1,72,72.000000
tone,1,61,61.000000
18,1,52,52.000000
guaranteed,1,51,51.000000
500,1,45,45.000000
cs,1,45,45.000000
1000,1,42,42.000000
awarded,1,39,39.000000


In [ ]:
# advanced: implement your own naive bayes classifier
